In [ ]:
binance_api_key = '<your api key here>'
binance_api_secret = '<your api secret here>'

In [ ]:
#this just makes the notebook take up most of your screen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pandas as pd
from numpy import pi
import numpy as np
import holoviews as hv
from coinmarketcap import Market
from datetime import datetime

from random import shuffle
from math import sin,cos
from bokeh.plotting import ColumnDataSource,output_notebook,figure, show
from bokeh.models import HoverTool,Text
from bokeh import palettes
import time

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.set_config_file(offline=True)
init_notebook_mode(connected=True)

from ipywidgets import interact
import ipywidgets as widgets

output_notebook()
hv.extension('bokeh', 'matplotlib')

from binance.client import Client
client = Client(binance_api_key, binance_api_secret)

In [ ]:
#this snippet usually resolves binance server timing issue
client = Client(binance_api_key, binance_api_secret)
int(time.time() * 1000) - client.get_server_time()['serverTime']

In [ ]:
#get account coin balances
order = client.get_account()
#convert list of dictionaries to pandas df
coin_df = pd.DataFrame(order['balances'])
#convert free and locked columns to numeric
coin_df[['free','locked']] = coin_df[['free','locked']].apply(pd.to_numeric)
#drop rows where all columns are less than or equal to 0
coin_df = coin_df.loc[~(coin_df<=0.0).all(axis=1)]
#add Total column to dataframe
coin_df['Total'] = coin_df['free'] + coin_df['locked']
#replace IOTA with MIOTA so that symbols match
coin_df.loc[coin_df.asset == 'IOTA', 'asset'] = 'MIOTA'

In [ ]:
coinmarketcap = Market()
prices = coinmarketcap.ticker(limit=1000, convert='USD')
prices_df = pd.DataFrame(prices)
#grabs prices from coinmarketcap
prices_df.head()

In [ ]:
full_df  = pd.merge(coin_df, prices_df, left_on='asset',right_on='symbol',how='inner')
full_df['price_usd'] = full_df['price_usd'].apply(pd.to_numeric)
full_df['percent_change_1h'] = full_df['percent_change_1h'].apply(pd.to_numeric)
full_df['percent_change_24h'] = full_df['percent_change_24h'].apply(pd.to_numeric)
full_df['percent_change_7d'] = full_df['percent_change_7d'].apply(pd.to_numeric)
full_df['estimated_holding_USD'] = full_df['price_usd'] * full_df['Total']
full_df = full_df.sort_values(by=['estimated_holding_USD'],ascending=False)
full_df = full_df[['asset', 'free', 'locked', 'Total', '24h_volume_usd',
       'available_supply', 'id', 'market_cap_usd',
       'max_supply', 'name', 'percent_change_1h', 'percent_change_24h',
       'percent_change_7d', 'price_btc', 'price_usd', 'rank', 'symbol',
       'total_supply', 'estimated_holding_USD']]

In [ ]:
class CustomPieBuilder:
    
    """
    custom pie chart builder for portfolio allocation, thanks to the random guy from stackoverflow I stole this from, it's great
    """
    green ="#50ee70"
    red = "#ff7070"
    x_range = 1.1
    y_range = 1.1

    def __init__(self,df,label_name,column_name,tools='hover',tooltips=None,
                 reverse_color=False,colors=None,random_color_order=False,
                 plot_width=400,plot_height=400,title='Untitled',*args,**kwargs):
        p = self.setup_figure(tools,plot_width,plot_height,title)
        df = self.add_columns_for_pie_chart(df,column_name,colors,reverse_color,random_color_order)
        self.df = df
        self.plot_pie(p,df,label_name,*args,**kwargs)
        if tooltips:
            self.set_hover_tooltip(p,tooltips)

        self.add_text_label_on_pie(p,df,label_name)
        self.plot = p

    def setup_figure(self,tools,plot_width,plot_height,title):
        p = figure(
            x_range=(-self.x_range, self.x_range),
            y_range=(-self.y_range, self.y_range),
            tools=tools,
            plot_width=plot_width,
            plot_height=plot_height,
            title=title,
        )
        p.axis.visible = False
        p.xgrid.grid_line_color = None
        p.ygrid.grid_line_color = None
        return p

    @staticmethod
    def plot_pie(p,df,label_name,*args,**kwargs):
        for key, _df in df.groupby(label_name):
            source = ColumnDataSource(_df.to_dict(orient='list'))
            p.annular_wedge(
                x=0,
                y=0,
                inner_radius=0,
                outer_radius=1,
                start_angle='starts',
                end_angle='ends',
                color='colors',
                source=source,
                legend=key,
                *args,**kwargs)

    @staticmethod
    def set_hover_tooltip(p,tooltips):
        hover = p.select({'type':HoverTool})
        hover.tooltips = tooltips

    @staticmethod
    def add_columns_for_pie_chart(df,column_name,colors=None,reverse_color=False,random_color_order=False):
        r = 0.7
        df = df.copy()
        column_sum = df[column_name].sum()
        df['percentage'] = (df[column_name]/column_sum)
        percentages = [0]  + df['percentage'].cumsum().tolist()
        df['starts'] = [p * 2 * pi for p in percentages[:-1]]
        df['ends'] = [p * 2 * pi for p in percentages[1:]]

        df['middle'] = (df['starts'] + df['ends'])/2
        df['text_x'] = df['middle'].apply(cos)*r
        df['text_y'] =df['middle'].apply(sin)*r 
        df['text_angle'] = 0.0

        if colors:
            df['colors'] = colors
        else:
            if 'colors' not in df:
                reverse_color = -1 if reverse_color else 1
                colors = palettes.viridis(len(df))[::reverse_color]
                if random_color_order:
                    shuffle(colors)
                df['colors'] = colors
        return df

    @staticmethod
    def add_text_label_on_pie(p,df,label_name):
        source=ColumnDataSource(df.to_dict(orient='list'))
        txt = Text(x="text_x", y="text_y", text=label_name, angle="text_angle",
               text_align="center", text_baseline="middle",
               text_font_size='10pt',)
        p.add_glyph(source,txt)

def build_plot(df,label_name,column_name,tools='hover',tooltips=None,
                 reverse_color=False,colors=None,random_color_order=False,
                 plot_width=400,plot_height=400,title='Untitled',*args,**kwargs):

    customPie = CustomPieBuilder(df,label_name,column_name,tools,tooltips,
                 reverse_color,colors,random_color_order,
                 plot_width,plot_height,title,*args,**kwargs)

    return customPie.plot

# Detailed summary of your holdings

In [ ]:
styled_full_df = full_df.style.bar(subset=['percent_change_1h', 'percent_change_24h','percent_change_7d'], align='mid', color=['#d65f5f', '#5fba7d'])
styled_full_df

# Portfolio Allocation

In [ ]:
p = build_plot(
    full_df,
    'symbol',
    'estimated_holding_USD',
    tooltips=[('percentage', '@percentage{0.00%}'), ('Coin', '@name'), ('Dollars','$'+'@estimated_holding_USD'),('Number Coins', '@Total')],
    title='Portfolio Allocation in USD',
    reverse_color=True,
    random_color_order=True,
    plot_height=900,
    plot_width=950)

print('Your total estimated holdings Binance in USD is: $',full_df['estimated_holding_USD'].sum())
show(p)

In [ ]:


def get_historical_price_df(cryptocurrency='iota', start_date = '20130428', end_date = datetime.now().strftime('%Y%m%d')):
    
    """gets historical prices and market cap of a particular cryptocurrency as a pandas df

        Args:
            cryptocurrency: full name of the cryptocurrency to pull

        Returns:
            historical_df: pandas df of historical data for the cryptocurrency
    """
    
    #we'll just take from 2013 up until current datetime
    url_string = "https://coinmarketcap.com/currencies/" + cryptocurrency + '/historical-data/?start=' + start_date + '&end=' + end_date
    historical_price_df = pd.read_html(url_string,parse_dates=['Date'])[0]
    historical_price_df = historical_price_df.set_index('Date')
    
    return historical_price_df

#get_historical_price_df('bitcoin-cash')

# Coin Plotting

### These are working graphs, still trying to figure out the best way to display this information

In [ ]:
#gets df of 1000 top coins
all_prices = coinmarketcap.ticker(limit=1000, convert='USD')
prices_df_all = pd.DataFrame(all_prices)
prices_df_all.head()

In [ ]:
choose_crypto = widgets.Dropdown(
    options=sorted(list(prices_df_all['id'])),
    value='bitcoin',
    description='Coin:',
    disabled=False,
)

In [ ]:
def dope_interactive_plot(crypto_chosen):
    """
    temp function to use for interactive plotting until DASH app is done
    """
    coin_OHLC_df = get_historical_price_df(crypto_chosen)
    coin_OHLC_df.iplot(kind='candle',theme='solar',up_color='green',down_color='red',rangeselector={ 'steps':['5y','1yr', '3 months','3 weeks','ytd','reset'], 'bgcolor' : ('grey',.3), 'x': 0.01 , 'y' : 0.99},title='Candle')
    
    mid_graph = cf.QuantFig(coin_OHLC_df,title='Test Quant Figure',legend='top',name='GS')
    mid_graph.add_sma([7,14],width=2,color=['darkblue','lightblue'],legendgroup=True)
    mid_graph.iplot(kind='candle',up_color='green',down_color='red',rangeslider=True,theme='solar',title='SMA w/ range slider')
    
    bottom_graph = cf.QuantFig(coin_OHLC_df,title='Test Quant Figure',legend='top',name='GS')
    bottom_graph.add_bollinger_bands(periods=7,boll_std=2,colors=['grey','magenta'],fill=True)
    bottom_graph.add_volume()
    bottom_graph.iplot(kind='ohlc',up_color='green',down_color='red',title='Bollanger Bands w/ Volume')

In [ ]:
interact(dope_interactive_plot,crypto_chosen=choose_crypto)

In [ ]:
def dope_interactive_plot2(crypto_chosen):
    """
    temp function to use for interactive plotting until DASH app is done, 2nd one b/c of iopub limit
    """
    coin_OHLC_df = get_historical_price_df(crypto_chosen)
    
    all_TA_graph = cf.QuantFig(coin_OHLC_df,title='Test Quant Figure',legend='top',name='GS')
    all_TA_graph.add_bollinger_bands(periods=7,boll_std=2,colors=['grey','magenta'],fill=True)
    all_TA_graph.add_macd()
    all_TA_graph.add_volume(color='magenta')
    all_TA_graph.add_rsi(periods=7,color='orange')
    all_TA_graph.iplot(kind='candle',up_color='green',down_color='red',title='Technical Indicators')

In [ ]:
interact(dope_interactive_plot2,crypto_chosen=choose_crypto)

In [ ]:
choose_crypto1 = widgets.Dropdown(
    options=sorted(list(prices_df_all['id'])),
    value='ethereum',
    description='Coin1:',
    disabled=False,
)
choose_crypto2 = widgets.Dropdown(
    options=sorted(list(prices_df_all['id'])),
    value='litecoin',
    description='Coin2:',
    disabled=False,
)

In [ ]:
def spread_chart_interactive(crypto_chosen1,crypto_chosen2):
    """
    temp function to use for interactive plotting of spread chart
    """
    coin_spread_df1 = get_historical_price_df(crypto_chosen1)
    coin_spread_df1['return'] = coin_spread_df1['Close'].pct_change(-1)

    coin_spread_df2 = get_historical_price_df(crypto_chosen2)
    coin_spread_df2['return'] = coin_spread_df2['Close'].pct_change(-1)

    return_spread_df = pd.concat([coin_spread_df1['return'], coin_spread_df2['return']], axis=1, keys=[crypto_chosen1, crypto_chosen2]).dropna()
    
    return_spread_df.iplot(kind='spread',xTitle='Dates',yTitle='Return',title='crypto returns Spread Chart',theme='solar')

In [ ]:
interact(spread_chart_interactive,crypto_chosen1=choose_crypto1,crypto_chosen2=choose_crypto2)

In [ ]:
choose_crypto_fit = widgets.Dropdown(
    options=sorted(list(prices_df_all['id'])),
    value='bitcoin',
    description='Coin:',
    disabled=False,
)

In [ ]:
def best_fit_interactive(crypto_chosen_fit):
    """
    temp function to use for interactive plotting of spread chart
    """
    coin_spread_df1 = get_historical_price_df(crypto_chosen_fit)
    coin_spread_df1['return'] = coin_spread_df1['Close'].pct_change(-1)
    newdf = coin_spread_df1['return'].dropna()
    newdf.iplot(kind='scatter',xTitle='Dates',yTitle='Returns',title='Daily Returns Besfit Line Chart',
                         filename='Returns - bestline fit chart',bestfit=True,colors=['blue'],
                         bestfit_colors=['pink'])
    newdf.iplot(kind='histogram',opacity=.75,title='Daily Returns Distribution')

In [ ]:
interact(best_fit_interactive,crypto_chosen_fit=choose_crypto_fit)

# Withdrawal History

In [ ]:
pd.DataFrame(client.get_withdraw_history()['withdrawList'])


# Deposit History

In [ ]:
pd.DataFrame(client.get_deposit_history()['depositList'])

# Full Trade History

In [ ]:
def typecolumn(df):
    """
    Simple helper function to create buy/sell column for trade history
    """
    if df['isBuyer'] == True:
        return 'Buy'
    else:
        return 'Sell'

In [ ]:
def convert_timestamp(timestamp):
    
    """
    helper to convert strade history timestamp column
    """
    if type(timestamp) == int:
        string_timestamp = str(timestamp)[:-3]
    else:
        string_timestamp = str(timestamp)[:-2]
        
    converted_timestamp = datetime.fromtimestamp(int(string_timestamp)).strftime('%Y-%m-%d %H:%M:%S')
    
    return converted_timestamp

convert_timestamp(1513289000001)

In [ ]:
#this cell gets your entire trade history
full_trades_df = []
#gets all possible tickers from binance
all_tickers = pd.DataFrame(client.get_all_tickers())

for i in all_tickers['symbol'].values:
    trades_df = pd.DataFrame(pd.DataFrame(client.get_my_trades(symbol=i)))
    trades_df['symbol'] = i
    full_trades_df.append(trades_df)
    
trade_history_df = pd.concat(full_trades_df)
trade_history_df['Type'] = trade_history_df.apply(typecolumn, axis=1)
#convert timestamp column to readable format
trade_history_df['time'] = pd.to_datetime(trade_history_df['time'], unit='ms')
trade_history_df = trade_history_df.sort_values(by='time',ascending=False)

In [ ]:
trade_history_df